<h1>Here Starts the College cleaning :_:</h1>

In [1]:
import pandas as pd
import numpy as np
import re
import dask.dataframe as dd
from dask.multiprocessing import get
from tqdm import tqdm

In [2]:
def syn_gen(col_name):
    lst = [None]*25
    if col_name == 'nan':
        return ['nan']
    col_name = str(col_name)
    col_name = col_name.lower()
    name = col_name.split(',')[0]
    names = []
    names.append(name)                                           # As given
    
    name = re.subn('of|\sand|&', '', name, flags = re.IGNORECASE)[0]
    
    name = re.subn('\s+', ' ', name)[0]                          # Removing 'of' and 'and'
    names.append(name)
    names.append(''.join([word[0] for word in name.split()]).upper())
    
    if "'" in col_name:    
        names.append(''.join([word[0] for word in filter(None,re.split('\W+', name))]).upper())
    if col_name.count('.') > 1:
        names.append(''.join(re.split(r'\.\s?', col_name)[:-1]).upper())
        
        try:
            pattern = re.compile('\s?univ\S*\s?|\s?coll\S*\s?|\s?inst\S*\s?|\sres\S*\sfound\S*\s?')
            inst_type = pattern.findall(name)[0]
            names.append(names[-1]+inst_type)
        except:
            pass
        
    if ',' in col_name:
        location_full = [name+re.findall(',.*', col_name)[0][1:] for name in names]
        try:
            location_short = [name+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)]
            location_short.extend([name+'-'+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)])
            location_short.extend([name+'/'+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)])
            location_short.extend([name+' '+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)])
            names.extend(location_short)
        except:
            pass
        names.extend(location_full)
    return (list(set(names))+lst)[:25]

In [3]:
def rm_brk(s):
    s = str(s)
    return re.sub('\\(.+?\\)','',s)

def ret_zero(lst):
    return lst[0]

def add_left_brk(s):
    s = str(s)
    s = s+')'
    return s

In [4]:
data = pd.read_csv('../data/all_institutions_live.csv', engine='python')
data['name_clean'] = data.name
data.iloc[208].name_clean

'100094-COLLEGE OF ARCHITECTURE, JALGAON'

In [5]:
data.name_clean = data.name_clean.apply(rm_brk)
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')


In [6]:
data.loc[data.name_clean.str.contains('\('),'name_clean']=data[data.name_clean.str.contains('\(')].name_clean.apply(add_left_brk)
data.name_clean = data.name_clean.apply(rm_brk)
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')
# data.loc[data.name_clean.str.contains('\('),'name_clean'].values

In [7]:
data['name_clean'] = data.name_clean.str.split('Ph:|PH[0-9]|Ph[0-9]|Ph. No|Ph\.|Mob\.No\.|PH\.|ph\.|Address|Dist.|Ward No|PB[0-9]|No [0-9]|PB [0-9]|Distt\.|Distt|H\.No|PIN-|PIN [0-9]|PIN -|Pin-|village|VILLAGE|Write institute Name here|near|Near|NEAR|DIST-|City Off|DIST:|DIST\.|D\.No|Gat No').apply(ret_zero)

data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

In [8]:
def rep_sub_none(string):
    string = str(string)
    return re.sub('[0-9]{3} [0-9]{3}-|[0-9]{6}-|Dist|"|\)','',string)

def rep_sub_quote(string):
    string = str(string)
    string = re.sub(',\.|PINCODE|PB |NH-[0-9]{1,4},|NH-[0-9]{1,4}|Nh-|NH[0-9]{1,4}|NH [0-9]|^[A-Z][0-9]{1,4} |^[A-Z]-[0-9]{1,4}|^[0-9]{1,4} |^[A-Z][0-9]{1,4} |^\?\?|^\?|\+3 Degree|\+3|\+ 3|^\+','',string)
    return re.sub(';|:|\?\?\?\?|\?\?\?|\?\?|\?','\'',string)

def rep_sub_fs(string):
    string = str(string)
    return re.sub('\.\.|>','.',string)

def rep_sub_ch(string):
    string = str(string)
    string =  re.sub('---|--','-',string)
    string = re.sub('  ,  |  , | ,  | , ',', ',string)
    return re.sub(',,,|,,|<',',',string)

def rep_digit(string):
    string = str(string)
    return re.sub('[0-9]{1,4}$|[0-9]{1,4}.$','',string)

In [9]:
data.name_clean = data.name_clean.apply(rep_sub_none)

data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')
data.iloc[208].name_clean

'COLLEGE OF ARCHITECTURE, JALGAON'

In [10]:
data['name_clean'] = data.name_clean.str.split('-[0-9]{3} [0-9]{3}|-[0-9]{6}|P\.O\.|P\.O|VPO|PB\.NO|- [0-9]{3} [0-9]{3}|- [0-9]{6}|[0-9]{3} [0-9]{3}|[0-9]{6}|Ta:|Di:|TAL:|OPP:|Vill|near|Post Box|P\.B\. No|Tq:|Dt:|Tq.:|Opp:|Opp\.|opp\.|Tal:|Di stt:|TQ:|At:|CITY:|Plot|PLOT|Opposite|Opp\.|OPP\.').apply(ret_zero)
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

data['name_clean'] = data.name_clean.str.split('P\.B\.No\.|P B No\.|No\.|PO |Post \?|Teh.|At post|Sector|SECTOR|sector|plot|Tal\.|Tehsil|DISTT|VILL-|VIll|VILL\.|NO\.|VILL |POST\.|TQ\.|OPP |OPP-|TAL-|TAL\.|AT\.|#|S NO|[0-9]{1,4}/|[0-9]{1,4}-').apply(ret_zero)
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

In [11]:
data.name_clean = data.name_clean.apply(rep_sub_quote)

data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

data.name_clean = data.name_clean.apply(rep_sub_fs)

data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

data.name_clean = data.name_clean.apply(rep_sub_ch)

data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

In [12]:
data.name_clean = data.name_clean.apply(rep_digit)

data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

In [13]:
data['name_clean'] = data.name_clean.str.split(' [A-Z]-[0-9]{1,4}|[0-9]{3,4}|N\.H\.|SECT\.|STREET NO-|[0-9]{1,4}th|[0-9]{1,4}rd|[0-9]{1,3} KM|At/,| [0-9]{1,3},').apply(ret_zero)
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.lstrip()
data.name_clean = data.name_clean.str.lstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('-')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('?')
data.name_clean = data.name_clean.str.rstrip('&')

In [14]:
data[['cid','name','name_clean']].to_csv('college_name_cleaned.csv',header=['cid','initial','cleaned'])

In [15]:
data[data.name_clean.str.contains('')].name_clean.values
# data.name_clean = data.name_clean.str.replace(',$','.')
# data.name_clean.

array(['Manish Institute of Computer Studies',
       'VIRANI SCI. & SHRI YOGIJI MAHARAJ ARTS, COMMERCE & SCIENCE - RAJKOT',
       'SIDDARTHA COLLEGE OF EDUCATION,  JAMKHANDI', ...,
       'Bharat Ratana Dr. B. R. Ambedkar University',
       'Delhi Technological University',
       'Guru Gobind Singh Indraprastha University'], dtype=object)

In [16]:
# data[data.name_clean.duplicated()].name_clean.shape
data[data.name.duplicated()].name.shape

(214,)

<h1>Here goes the city cleaning :-) </h1>

<h2>Remember to remove below read data</h2>

In [958]:
data = pd.read_csv('../data/all_institutions_live.csv', engine='python')
data['name_clean'] = data.name

In [959]:
data['city_clean'] = data.city
data.loc[8854,'city_clean']

'Bangalore-10.'

In [960]:
data.city_clean = data.city_clean.str.lower()

In [961]:
sum(data.city_clean.isnull())

1816

In [962]:
data.city_clean.fillna('NULL',inplace=True)
data.loc[data.city_clean.str.contains('^[0-9]'),'city_clean']='NULL'

In [963]:
data.city_clean = data.city_clean.str.rstrip()
data.city_clean = data.city_clean.str.rstrip(',')
data.city_clean = data.city_clean.str.rstrip('.')
data.city_clean = data.city_clean.str.rstrip('-')
data.city_clean = data.city_clean.str.lstrip()

In [964]:
data['city_clean'] = data.city_clean.str.split('pin| dist|,dist').apply(ret_zero)
data['city_clean'] = data.city_clean.str.split('[0-9]{3} [0-9]{3}|[0-9]{6}|[0-9]{1,5}$').apply(ret_zero)

In [965]:
def make_none_first(string):
    string = str(string)
    return re.sub('po & mandal:|tq & dt|kustagi road,|tk & dt|first lane, |at post -peeth ,|at post tha|at & po\.|at & post|at post','',string)

data.city_clean = data.city_clean.apply(make_none_first)

data.city_clean = data.city_clean.str.lstrip()
data.city_clean = data.city_clean.str.lstrip('-')
data.city_clean = data.city_clean.str.lstrip(':')
data.city_clean = data.city_clean.str.lstrip('.')
data.city_clean = data.city_clean.str.lstrip(',')
data.city_clean = data.city_clean.str.lstrip()
data.city_clean = data.city_clean.str.rstrip()
data.city_clean = data.city_clean.str.rstrip(',')
data.city_clean = data.city_clean.str.rstrip('.')
data.city_clean = data.city_clean.str.rstrip('-')
data.city_clean = data.city_clean.str.rstrip()

In [966]:
data.loc[14655,'city_clean']='chandigarh'
data.loc[1960,'city_clean']='etawah'
data.loc[11580,'city_clean']='vijayawada'
data.loc[35751,'city_clean']='surat'
data.loc[35850,'city_clean']='patan'
data.loc[508,'city_clean']='kurukshetra'
data.loc[21425,'city_clean']='banglore'
data.loc[21756,'city_clean']='sangaria'
data.loc[43721,'city_clean']='perinthalmanna'
data.loc[21587,'city_clean']='murudeshwar'
data.loc[38468,'city_clean']='salt lake city'
data.loc[10529,'city_clean']='nellimarla'
data.loc[28797,'city_clean']='atraulihar'
data.loc[42918,'city_clean']='bareilly'
data.loc[28687,'city_clean']='jambusar'
data.loc[27548,'city_clean']=data.loc[12756,'city_clean']='linch'
data.loc[35077,'city_clean']='virpur'
data.loc[36253,'city_clean']='naigaon'
data.loc[44812,'city_clean']='kanakapura'
data.loc[39102,'city_clean']='neyveli township'


In [967]:
data['city_clean'] = data.city_clean.str.split(' village|opp. | mandal| tehsil| via| near| taluka| taluk| tal | tal:| tal-| tal\.| tah| ta | ta:| ta-| ta\.| tq|\.tq').apply(ret_zero)
data.city_clean = data.city_clean.str.rstrip()
data.city_clean = data.city_clean.str.rstrip(',')
data.city_clean = data.city_clean.str.rstrip('.')
data.city_clean = data.city_clean.str.rstrip('-')
data.city_clean = data.city_clean.str.rstrip()
# ^ta-|^ta:|^tal:-|^tal-|^tal\.|^ta\.:-|^ta\.|^ta\.:|

In [968]:
def make_none(string):
    string = str(string)
    return re.sub('district -|district-|district:|dictrict|district|distt-|distt\. -|distt \.|distt\.|distt -|distt:|distt :|distt|dist-|dist -|dist\.|dist \.|dist:|dist :|dist|[0-9]|junction|village,|village\.|village-|village|villl|vill&mdl|vill-|vill:|vill\.|villege|vill |mandal:|mandal |tehsil-|tehsil -|tehsil|teh |teh\.|teh-|teh:|vpo|via |via\.|via-|via|near by|near|taluka:|taluka-|taluka\.|taluka |taluka|\.taluk|-taluk|taluk|tal |tal:|tal-|tal$|talik$|talk$|tal\.|tah |tah-|^ta|tq|dt$|tk$|^town|town$|suburban|urban|rural|ps |ps-|p\.s |p\.s\.|^at |post$|post-|post |post:| up$','',string)

In [969]:
data.city_clean = data.city_clean.apply(rm_brk)
data.city_clean = data.city_clean.str.rstrip()
data.city_clean = data.city_clean.str.rstrip(',')
data.city_clean = data.city_clean.str.rstrip('.')
data.city_clean = data.city_clean.str.rstrip('-')
data.city_clean = data.city_clean.str.rstrip()

In [970]:
data.city_clean = data.city_clean.apply(make_none)
data.city_clean = data.city_clean.str.lstrip()
data.city_clean = data.city_clean.str.lstrip()
data.city_clean = data.city_clean.str.lstrip()
data.city_clean = data.city_clean.str.lstrip('-')
data.city_clean = data.city_clean.str.lstrip('-')
data.city_clean = data.city_clean.str.lstrip('-')
data.city_clean = data.city_clean.str.lstrip(':')
data.city_clean = data.city_clean.str.lstrip(':')
data.city_clean = data.city_clean.str.lstrip('.')
data.city_clean = data.city_clean.str.lstrip('.')
data.city_clean = data.city_clean.str.lstrip(',')
data.city_clean = data.city_clean.str.lstrip(',')
data.city_clean = data.city_clean.str.lstrip()
data.city_clean = data.city_clean.str.lstrip()
data.city_clean = data.city_clean.str.lstrip()

In [971]:
data.city_clean = data.city_clean.str.rstrip()
data.city_clean = data.city_clean.str.rstrip()
data.city_clean = data.city_clean.str.rstrip(',')
data.city_clean = data.city_clean.str.rstrip(',')
data.city_clean = data.city_clean.str.rstrip('.')
data.city_clean = data.city_clean.str.rstrip('.')
data.city_clean = data.city_clean.str.rstrip('-')
data.city_clean = data.city_clean.str.rstrip('-')
data.city_clean = data.city_clean.str.rstrip()
data.city_clean = data.city_clean.str.rstrip()

In [972]:
data[data.city_clean.str.contains(' up$')].city_clean

Series([], Name: city_clean, dtype: object)

In [973]:
data.loc[data.city_clean=='','city_clean'] = 'NULL'

In [974]:
len(data.city_clean.unique())

7423

In [977]:
data[['city','city_clean']].to_csv('city_comp.csv')

<h1>Synonyms generation</h1>

In [173]:
len(names.iloc[4]['name'])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [174]:
names.name.values.tolist()
names[list(range(25))] = pd.DataFrame(names.name.values.tolist(), index= names.index)

In [178]:
names['name']=data['name']

In [179]:
def comma(s):
    if str(s) != str(None):
        return '"'+s+'"'
    return None

In [180]:
names = names.apply(comma)

In [181]:
names[names[24].notnull()]

,name,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24


In [182]:
names = names.drop([24],axis=1)

In [183]:
names.to_csv('clg_entities.csv',header=None,index=None)

rm_brk('kunal')

In [157]:
rm_brk('kuna(kd)l')

'kunal'

In [166]:
data['name'] = data['name'].apply(rm_brk)

In [185]:
data.to_csv('all_clgs.csv')

In [186]:
names.head()

,name,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,"""Manish Institute of Computer Studies""","""MICS""","""manish institute of computer studies""","""manish institute computer studies""",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"""SIDDARTHA COLLEGE OF EDUCATION, JAMKHANDI""","""SCE/J""","""siddartha college education""","""SCE jamkhandi""","""SCE-J""","""siddartha college of education""","""siddartha college education jamkhandi""","""siddartha college of education jamkhandi""","""SCE""","""SCE J""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"""VVS'S ARTS/COMM COLLEGE, BIJAPUR""","""VAC""","""VSACCB""","""VSACC bijapur""","""VAC-B""","""VAC B""","""VSACC/B""","""VSACC B""","""vvs's arts/comm college bijapur""","""VAC bijapur""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"""GOVT. FIRST GRADE COLLEGE, KOKATANOOR""","""GFGC-K""","""GFGC/K""","""GFGC K""","""GFGC kokatanoor""","""govt. first grade college kokatanoor""","""govt. first grade college""","""GFGC""","""GFGCK""",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"""S S S S'S COLLEGE OF BUSINESS ADMINISTRATION,...","""s s s s's college business administration be...","""SSSSCBA B""","""s s s s's college of business administration ...","""SSSSCBA/B""","""SSSSSCBA/B""","""SSSSCBA belgaum""","""SSSSCBA-B""","""SSSSSCBA""","""s s s s's college of business administration""",...,"""s s s s's college business administration""","""SSSSSCBA-B""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


college_entity.ipynb               merger.ipynb
college_front_page_scrapper.ipynb  scrapper2.ipynb
entity_generator.ipynb             temp_mapper.ipynb
entity_synonym_generator.ipynb     univ_names
mapper.ipynb


In [5]:
data = pd.read_csv('CollegeName_entity.csv')

In [10]:
data.iloc[:15000].to_csv('CollgeName1',header=None,index=None)

In [11]:
data.iloc[15000:].to_csv('CollegeName2',header=None,index=None)